# 1. Import dependencies

In [1]:
import gymnasium as gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [2]:
environment_name = "CartPole-v0"

In [3]:
env = gym.make(environment_name, render_mode="human")

C:\Users\srava\AppData\Roaming\Python\Python311\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        outcome = env.step(action)
        n_state = outcome[0]
        reward = outcome[1]
        done = outcome[2]
        info = outcome[3]
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:24.0
Episode:2 Score:38.0
Episode:3 Score:16.0
Episode:4 Score:18.0
Episode:5 Score:16.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [5]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

0

In [6]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 2.2481203e+00, -1.4837124e+38, -2.4155457e-01,  2.4513941e+37],
      dtype=float32)

# 3. Train an RL Model

In [7]:
env = gym.make(environment_name,  render_mode="human")
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


In [8]:
model.learn(total_timesteps=20)

-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 44   |
|    total_timesteps | 2048 |
-----------------------------


# 4. Save and Reload Model

In [9]:
import os
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [10]:
model.save(PPO_path)

In [11]:
del model

In [12]:
model = PPO.load('PPO_model', env=env)

# 5. Evaluation

In [13]:
from stable_baselines3.common.evaluation import evaluate_policy

In [14]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\srava\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [15]:
env.close()

# 6. Adding a callback to the training Stage

In [16]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [17]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [18]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

C:\Users\srava\AppData\Roaming\Python\Python311\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [19]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [20]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [21]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_8
-----------------------------
| time/              |      |
|    fps             | 1184 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 853         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008117784 |
|    clip_fraction        | 0.0979      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00255    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.87        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.015      |
|    value_loss           | 56.5        |
-----------------------------------------
---

In [22]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [23]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\srava\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(193.1, 17.90223449740283)

In [24]:
env.close()

# 7. Changing Policies

In [25]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [26]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


C:\Users\srava\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [27]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 1378 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 851         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014235102 |
|    clip_fraction        | 0.202       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | 0.00304     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.03        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0206     |
|    value_loss           | 19.4        |
-----------------------------------------
----------------------------------

# 8. Using an Alternate Algorithm

In [28]:
from stable_baselines3 import DQN

In [29]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [30]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_8
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.971    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4906     |
|    time_elapsed     | 0        |
|    total_timesteps  | 61       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.927    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 6197     |
|    time_elapsed     | 0        |
|    total_timesteps  | 153      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.885    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 8724     |
|    time_elapsed     | 0        |
|    total_timesteps  | 242      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 8639     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2429     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 8629     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2507     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 8925     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2611     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 8425     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4850     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 8477     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4947     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 8475     |
|    time_elapsed     | 0        |
|    total_timesteps  | 5011     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 8722     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7237     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 8701     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7337     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 8699     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7415     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 8671     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9649     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 8670     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9765     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 8694     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9840     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 8490     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11823    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 8496     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11910    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 8506     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11976    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 8407     |
|    time_elapsed     | 1        |
|    total_timesteps  | 13963    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 8406     |
|    time_elapsed     | 1        |
|    total_timesteps  | 14059    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 8432     |
|    time_elapsed     | 1        |
|    total_timesteps  | 14152    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 8538     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16563    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 8585     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16697    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 8570     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16798    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 8660     |
|    time_elapsed     | 2        |
|    total_timesteps  | 18962    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 8696     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19084    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 8670     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19150    |
----------------------------------
----------------------------------
| rollout/          

In [31]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [32]:
model.save(dqn_path)

In [33]:
model = DQN.load(dqn_path, env=env)

In [34]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(12.3, 1.676305461424021)

In [35]:
env.close()